# Making pipeline for data cleaning and preparation for sklearn models

In [1]:
# main imports
import importlib # for reloading custom libraries

from datetime import datetime, timedelta

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.exceptions import NotFittedError

from sklearn.ensemble import RandomForestRegressor # model to test resulting dataset on

from sklearn.base import BaseEstimator, TransformerMixin # base classes to create custom transformers

# custom transformers based on sklearn library
import pipeline_classes as plc
#from pipeline_classes import DFCreateAdditionalFeatures, DFReplaceMeaningfulNANs, DFJoinDates, DFCalcAge, DFDropColumns, DFConvertToNumpy, DFOneHotCategoriesCombined

In [2]:
from sklearn.metrics import mean_squared_log_error

In [3]:
# if changes were made to pipeline_classes.py
#importlib.reload(plc)

## Loading original data to be processed

In [4]:
df_orig = pd.read_csv('datasets/train.csv', index_col=0)
df_orig.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

'df' will be the name of dataset we are working with. All the versions we save as backups or milestones will have the suffix starting with '_'. This suffix with the cell comment will help us to recover what this data snapshot is needed for

In [5]:
df = df_orig.copy()

## Making some definitions

In [6]:
# Prints NANs numbers in columns if any
def print_nans_stat(df):
    print(print(f'Column \tNANs'))
    total = 0
    for c in df.columns:
        n = df[c].isna().sum()
        if n > 0:
            total += n
            print(f'{c} \t{n}')
    print('-'*20, f'\nTotal \t{total}')

In [7]:
# Categorical columns which will share same columns after transformed to One-Hot form (usually contain multiple characteristics of same type)
ohe_combined_columns = [['Condition1', 'Condition2'], ['Exterior1st', 'Exterior2nd']]

# Categorical columns which can be transformed to One-Hit form by themselves (do not require special processing)
ohe_single_columns = [
    'MSSubClass', 'MSZoning', 'Alley', 'LandContour', 'LotConfig', 'Neighborhood', 'BldgType', 'HouseStyle', 'RoofStyle', # 'Street' deleted
    'RoofMatl', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'Electrical', 'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition'
]

# Categorical columns which can be transformed to numeric form with defined space between categories (ordered categories)
ore_columns = [
    'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC',
    'LotShape', 'LandSlope', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Functional', 'GarageFinish', 'PavedDrive', 'Fence' # 'Utilities' deleted
]

# Numeric columns
num_columns = []
for column in df_orig.columns:
    if (df_orig[column].dtype != 'object') and (df_orig[column].dtype != 'category') and (column not in ohe_single_columns+ore_columns):
        num_columns.append(column)
num_columns.remove('SalePrice')

# Print checksums
print('Check total number of columns:')
print('Categorical unordered combined columns = 4')
print('Categorical unordered single columns = {}'.format(len(ohe_single_columns)))
print('Categorical ordered single columns = {}'.format(len(ore_columns)))
print('Numerical columns = {}'.format(len(num_columns)))
print('-'*20, '\nTOTAL = {}'.format(4 + len(ohe_single_columns) + len(ore_columns) + len(num_columns)))
print('Columns in original dataset: {}'.format(len(df_orig.columns)))

Check total number of columns:
Categorical unordered combined columns = 4
Categorical unordered single columns = 19
Categorical ordered single columns = 19
Numerical columns = 35
-------------------- 
TOTAL = 77
Columns in original dataset: 80


Here we have to take into account that 'Street' and 'Utilities' features will be deleted as bad regressors. And original dataset contains 'SalePrice' column which is prediction target so we removed it from 'num_columns'

## Dataset columns modifications

Let's recap transformations we will make with the train dataset:  
  
Get rid of 'Utilities' and 'Street' features  
Convert MSSubClass values to string type to avoid mixing string and numeric values in categories

Default values to replace nans with are 'Abs' for categorical features and 0 for numerical except years and months. For month and year they will be 1 and 1900 respectively.  

(1) There are several groups of "linked" fields where nans will be marks of physical absence of the feature instead of missing data. They are:  
'BASEMENT' section with linked features: ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']  
*Note: for 'BsmtFinType2' feature we have to separate if there is no basement and if basement exists but there is no "type2" (only "type1")  
  
'GARAGE' section with linked features: ['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond']  
'FIREPLACE' section with linked features: ['FireplaceQu', 'Fireplaces']  
'POOL' section with linked features: ['PoolQC', 'PoolArea']  
'MASONRY' section with linked features: ['MasVnrType', 'MasVnrArea'] (default nan replacement for 'MasVnrType' will be 'None')  
'MISC' section with linked features: ['MiscFeature', 'MiscVal']  
  
For these sections we are going to check if all the fields have nan values (or their equivalents). If this is the case we have to replace all nans with values marking absence (0 for numeric, 'Abs' for categorical fields).  
  
(2) There are single columns that can contain nans meaning absence of smth:  
Street, Alley, Fence  
  
(3) All other nans should be treated as real missing values  
  
(4) There are columns that should be treated as containing ordered categories (quality measured):  
['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']  
  
They should be processed using OrdinalEncoder()  
  
(5) There are columns that should be treated as containing UNORDERED categories  
  
They should be transformed to one-hot features.  
But there are pairs of features meaning multiple attributes of the same object: [['Condition1', 'Condition2'], ['Exterior1st', 'Exterior2nd']]. They should share same one-hot columns for each pair

In [8]:
pd.crosstab(df['BsmtQual'], df['BsmtFinType2'], dropna=False)

BsmtFinType2,ALQ,BLQ,GLQ,LwQ,Rec,Unf
BsmtQual,,,,,,
Ex,2,0,2,0,0,117
Fa,0,0,0,1,0,34
Gd,5,9,9,17,10,567
TA,12,24,3,28,44,538


In [9]:
pd.crosstab(df['BsmtFinType1'], df['BsmtFinType2'], dropna=False)

BsmtFinType2,ALQ,BLQ,GLQ,LwQ,Rec,Unf
BsmtFinType1,,,,,,
ALQ,0,15,2,15,22,166
BLQ,2,1,5,13,15,112
GLQ,4,2,0,10,9,392
LwQ,9,4,7,0,8,46
Rec,4,11,0,8,0,110
Unf,0,0,0,0,0,430


In [10]:
print_nans_stat(df)

Column 	NANs
None
LotFrontage 	259
Alley 	1369
MasVnrType 	8
MasVnrArea 	8
BsmtQual 	37
BsmtCond 	37
BsmtExposure 	38
BsmtFinType1 	37
BsmtFinType2 	38
Electrical 	1
FireplaceQu 	690
GarageType 	81
GarageYrBlt 	81
GarageFinish 	81
GarageQual 	81
GarageCond 	81
PoolQC 	1453
Fence 	1179
MiscFeature 	1406
-------------------- 
Total 	6965


## Check for unexpected nans in test dataset

Getting test data to understand what features may come to data processing pipeline with nans

In [11]:
df_test = pd.read_csv('datasets/test.csv', index_col=0)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 1461 to 2919
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   int64  
 1   MSZoning       1455 non-null   object 
 2   LotFrontage    1232 non-null   float64
 3   LotArea        1459 non-null   int64  
 4   Street         1459 non-null   object 
 5   Alley          107 non-null    object 
 6   LotShape       1459 non-null   object 
 7   LandContour    1459 non-null   object 
 8   Utilities      1457 non-null   object 
 9   LotConfig      1459 non-null   object 
 10  LandSlope      1459 non-null   object 
 11  Neighborhood   1459 non-null   object 
 12  Condition1     1459 non-null   object 
 13  Condition2     1459 non-null   object 
 14  BldgType       1459 non-null   object 
 15  HouseStyle     1459 non-null   object 
 16  OverallQual    1459 non-null   int64  
 17  OverallCond    1459 non-null   int64  
 18  YearB

Calculate nans in train and test dataset columns to understand places where unexpected nans may occure.  
We have to deal with nan values in train set in one way or another. So here we are interested in cases when there is no nans in train set but there are some in test set 

In [12]:
nans_train = {}
nans_test = {}
for c in df_test.columns:
    n = df_test[c].isna().sum()
    if n > 0:
        nans_test[c] = n
    n = df[c].isna().sum()
    if n > 0:
        nans_train[c] = n

print('Column \tTrain set \tTest set')
for c, v in nans_test.items():
    if nans_train.get(c, 0) == 0:
        print('{} \t{} \t{}'.format(c, nans_train.get(c, 0), v))


Column 	Train set 	Test set
MSZoning 	0 	4
Utilities 	0 	2
Exterior1st 	0 	1
Exterior2nd 	0 	1
BsmtFinSF1 	0 	1
BsmtFinSF2 	0 	1
BsmtUnfSF 	0 	1
TotalBsmtSF 	0 	1
BsmtFullBath 	0 	2
BsmtHalfBath 	0 	2
KitchenQual 	0 	1
Functional 	0 	2
GarageCars 	0 	1
GarageArea 	0 	1
SaleType 	0 	1


In [13]:
pd.crosstab(df['Exterior1st'], df['Exterior2nd'], dropna=False)

Exterior2nd,AsbShng,AsphShn,Brk Cmn,BrkFace,CBlock,CmentBd,HdBoard,ImStucc,MetalSd,Other,Plywood,Stone,Stucco,VinylSd,Wd Sdng,Wd Shng
Exterior1st,,,,,,,,,,,,,,,,
AsbShng,17,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0
AsphShn,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BrkComm,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0
BrkFace,1,0,0,24,0,0,3,0,0,0,6,2,1,0,12,1
CBlock,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
CemntBd,0,0,0,0,0,59,0,0,0,0,0,0,0,0,1,1
HdBoard,0,1,0,0,0,0,193,2,1,0,23,0,0,0,1,1
ImStucc,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
MetalSd,0,1,0,0,0,0,3,0,212,0,0,0,1,0,2,1


So to be able to deal with nans in these column we suggest the follows:  
- MSZoning - use fillna() with most frequent value  
- Utilities - will be deleted  
- Exterior1st and Exterior2nd - if one of the values is not NAN, use the other value, otherwise use fillna() with most frequent value 'VinylSd' (same for both)  
- 'Bsmt' features - should be handled in 'Basement' section  
- KitchenQual - use fillna() with most frequent ('TA') value  
- Functional - use fillna() with 'Typ' value (as mentioned in data_description file)  
- 'Garage' features - should be handled in 'Garage' section  
- SaleType - use fillna() with most frequent ('WD') value  

## Check steps for data cleaning logic

### Step 1 - Drop columns which are not used 

In [14]:
y = df_orig['SalePrice'].to_numpy()

In [15]:
df = plc.DFDropColumns(columns=['SalePrice', 'Utilities', 'Street']).fit_transform(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 77 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Alley          91 non-null     object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   LotConfig      1460 non-null   object 
 8   LandSlope      1460 non-null   object 
 9   Neighborhood   1460 non-null   object 
 10  Condition1     1460 non-null   object 
 11  Condition2     1460 non-null   object 
 12  BldgType       1460 non-null   object 
 13  HouseStyle     1460 non-null   object 
 14  OverallQual    1460 non-null   int64  
 15  OverallCond    1460 non-null   int64  
 16  YearBuilt      1460 non-null   int64  
 17  YearRemodAdd   1460 non-null   int64  
 18  RoofStyl

### Step 2 - Convert number categories in 'MSSubClass' to string type

In [16]:
df = plc.DFApplyFuncToColumns(columns={'MSSubClass': str}).fit_transform(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 77 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   object 
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Alley          91 non-null     object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   LotConfig      1460 non-null   object 
 8   LandSlope      1460 non-null   object 
 9   Neighborhood   1460 non-null   object 
 10  Condition1     1460 non-null   object 
 11  Condition2     1460 non-null   object 
 12  BldgType       1460 non-null   object 
 13  HouseStyle     1460 non-null   object 
 14  OverallQual    1460 non-null   int64  
 15  OverallCond    1460 non-null   int64  
 16  YearBuilt      1460 non-null   int64  
 17  YearRemodAdd   1460 non-null   int64  
 18  RoofStyl

In [17]:
default_nan_replacement = {  # Default values to replace NaNs with
        'MSZoning': df['MSZoning'].value_counts().index[0], # should be 'RL'
        'Alley': 'Abs',
        'Fence': 'Abs',
        'Electrical': df['Electrical'].value_counts().index[0], # should be 'SBrkr'
        # Basement section
        'BsmtQual': 'Abs', 'BsmtCond': 'Abs', 'BsmtExposure': 'Abs',
        'BsmtFinType1': 'Abs', 'BsmtFinSF1': 0, 'BsmtFinType2': 'Abs',
        'BsmtFinSF2': 0, 'BsmtUnfSF': 0, 'TotalBsmtSF': 0, 'BsmtFullBath': 0, 'BsmtHalfBath': 0,
        # Garage section
        'GarageType': 'Abs', 'GarageYrBlt': 0, 'GarageFinish': 'Abs', 'GarageCars': 0,
        'GarageArea': 0, 'GarageQual': 'Abs', 'GarageCond': 'Abs',
        # Other sections
        'FireplaceQu': 'Abs', 'Fireplaces': 0,
        'PoolQC': 'Abs', 'PoolArea': 0,
        'MiscFeature': 'Abs', 'MiscVal': 0,
        'MasVnrType': 'None', 'MasVnrArea': 0,
        'Exterior1st': 'VinylSd', 'Exterior2nd': 'VinylSd',
        'KitchenQual': 'TA',
        'Functional': 'Typ',
        'SaleType': df['SaleType'].value_counts().index[0], # should be 'WD'
    }

In [18]:
single_features = ['Alley', 'Fence', 'Electrical']
chained_features = {
    'BsmtQual': ['BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'],
    'GarageType': ['GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond'],
    'FireplaceQu': ['Fireplaces'],
    'PoolQC': ['PoolArea'],
    'MiscFeature': ['MiscVal'],
    'MasVnrType': ['MasVnrArea'],
}

In [19]:
cat_unordered_features = {
    'MSSubClass': pd.CategoricalDtype(categories=['Abs', '20', '30', '40', '45', '50', '60', '70', '75', '80', '85', '90', '120', '150', '160', '180', '190'], ordered=False),
    'MSZoning': pd.CategoricalDtype(categories=['Abs', 'A', 'C (all)', 'FV', 'I', 'RH', 'RL', 'RP', 'RM'], ordered=False), # 'C (all)' took from dataframe
    #'Street': pd.CategoricalDtype(categories=['Abs', 'Grvl', 'Pave'], ordered=False), # 'Street' feature deleted
    'Alley': pd.CategoricalDtype(categories=['Abs', 'Grvl', 'Pave'], ordered=False),
    'LandContour': pd.CategoricalDtype(categories=['Abs', 'Lvl', 'Bnk', 'HLS', 'Low'], ordered=False),
    'LotConfig': pd.CategoricalDtype(categories=['Abs', 'Inside', 'Corner', 'CulDSac', 'FR2', 'FR3'], ordered=False),
    'Neighborhood': pd.CategoricalDtype(categories=['Abs', 'Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr', 'Crawfor', 'Edwards', 'Gilbert',
                                                        'IDOTRR', 'MeadowV', 'Mitchel', 'NAmes', 'NoRidge', 'NPkVill', 'NridgHt', 'NWAmes', 'OldTown', 'SWISU',
                                                        'Sawyer', 'SawyerW', 'Somerst', 'StoneBr', 'Timber', 'Veenker'], ordered=False),
    'Condition1': pd.CategoricalDtype(categories=['Abs', 'Norm', 'Artery', 'Feedr', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe'], ordered=False), # Norm value is default and can
    'Condition2': pd.CategoricalDtype(categories=['Abs', 'Norm', 'Artery', 'Feedr', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe'], ordered=False), # Norm value is default and can
    'BldgType': pd.CategoricalDtype(categories=['Abs', '1Fam', '2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'], ordered=False), # Changed '2FmCon'=>'2fmCon', 'Duplx'=>'Duplex', 'TwnhsI'=>'Twnhs'
    'HouseStyle': pd.CategoricalDtype(categories=['Abs', '1Story', '1.5Fin', '1.5Unf', '2Story', '2.5Fin', '2.5Unf', 'SFoyer', 'SLvl'], ordered=False),
    'RoofStyle': pd.CategoricalDtype(categories=['Abs', 'Flat', 'Gable', 'Gambrel', 'Hip', 'Mansard', 'Shed'], ordered=False),
    'RoofMatl': pd.CategoricalDtype(categories=['Abs', 'ClyTile', 'CompShg', 'Membran', 'Metal', 'Roll', 'Tar&Grv', 'WdShake', 'WdShngl'], ordered=False),
    'Exterior1st': pd.CategoricalDtype(categories=['Other', 'AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock', 'CemntBd', 'HdBoard', 'ImStucc',
                                                        'MetalSd', 'Plywood', 'PreCast', 'Stone', 'Stucco', 'VinylSd', 'Wd Sdng', 'WdShing'], ordered=False),
    'Exterior2nd': pd.CategoricalDtype(categories=['Other', 'AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock', 'CemntBd', 'HdBoard', 'ImStucc',
                                                        'MetalSd', 'Plywood', 'PreCast', 'Stone', 'Stucco', 'VinylSd', 'Wd Sdng', 'WdShing'], ordered=False),
    'MasVnrType': pd.CategoricalDtype(categories=['BrkCmn', 'BrkFace', 'CBlock', 'None', 'Stone'], ordered=False),
    'Foundation': pd.CategoricalDtype(categories=['Abs', 'BrkTil', 'CBlock', 'PConc', 'Slab', 'Stone', 'Wood'], ordered=False),
    'Heating': pd.CategoricalDtype(categories=['Abs', 'Floor', 'GasA', 'GasW', 'Grav', 'OthW', 'Wall'], ordered=False),
    'CentralAir': pd.CategoricalDtype(categories=['Abs', 'N', 'Y'], ordered=False),
    'Electrical': pd.CategoricalDtype(categories=['Abs', 'SBrkr', 'FuseA', 'FuseF', 'FuseP', 'Mix'], ordered=False),
    'GarageType': pd.CategoricalDtype(categories=['Abs', '2Types', 'Attchd', 'Basment', 'BuiltIn', 'CarPort', 'Detchd'], ordered=False), # NA = No garage => 'Abs'
    'MiscFeature': pd.CategoricalDtype(categories=['Abs', 'Elev', 'Gar2', 'Othr', 'Shed', 'TenC'], ordered=False), # NA = None
    'SaleType': pd.CategoricalDtype(categories=['Abs', 'WD', 'CWD', 'VWD', 'New', 'COD', 'Con', 'ConLw', 'ConLI', 'ConLD', 'Oth'], ordered=False),
    'SaleCondition': pd.CategoricalDtype(categories=['Abs', 'Normal', 'Abnorml', 'AdjLand', 'Alloca', 'Family', 'Partial'], ordered=False),
}

In [20]:
quality_categories = pd.CategoricalDtype(categories=['Abs', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], ordered=True)

cat_ordered_features = {
    'ExterQual': quality_categories,
    'ExterCond': quality_categories,
    'BsmtQual': quality_categories,
    'BsmtCond': quality_categories,
    'HeatingQC': quality_categories,
    'KitchenQual': quality_categories,
    'FireplaceQu': quality_categories,
    'GarageQual': quality_categories,
    'GarageCond': quality_categories,
    'PoolQC': quality_categories,
    'LotShape': pd.CategoricalDtype(categories=['Abs', 'IR3', 'IR2', 'IR1', 'Reg'], ordered=True),
    'LandSlope': pd.CategoricalDtype(categories=['Abs', 'Sev', 'Mod', 'Gtl'], ordered=True),
    'BsmtExposure': pd.CategoricalDtype(categories=['Abs', 'No', 'Mn', 'Av', 'Gd'], ordered=True),
    'BsmtFinType1': pd.CategoricalDtype(categories=['Abs', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'], ordered=True),
    'BsmtFinType2': pd.CategoricalDtype(categories=['Abs', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'], ordered=True),
    'Functional': pd.CategoricalDtype(categories=['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'], ordered=True),
    'GarageFinish': pd.CategoricalDtype(categories=['Abs', 'Unf', 'RFn', 'Fin'], ordered=True),
    'PavedDrive': pd.CategoricalDtype(categories=['N', 'P', 'Y'], ordered=True),
    'Fence': pd.CategoricalDtype(categories=['Abs', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv'], ordered=True),
}

### Step 3 - Fix some errors in words typing

In 'Exterior2nd' we have to replace 'CmentBd' values with 'CemntBd' (like in 'Exterior1st' feature)  
In 'Exterior2nd' we have to replace 'Wd Shng' values with 'Wd Sdng' or 'WdShing' (in data_description there are 'Wd Sdng' and 'WdShing') - TRY BOTH  
In 'Exterior2nd' we have to replace 'Brk Cmn' values with 'BrkComm' (like in 'Exterior1st' feature)

In [21]:
df = plc.DFValuesReplacer(replaces={'Exterior2nd': {'CmentBd': 'CemntBd', 'Brk Cmn': 'BrkComm', 'Wd Shng': 'WdShing'}}).fit_transform(df)
pd.crosstab(df['Exterior1st'], df['Exterior2nd'], dropna=False)

Exterior2nd,AsbShng,AsphShn,BrkComm,BrkFace,CBlock,CemntBd,HdBoard,ImStucc,MetalSd,Other,Plywood,Stone,Stucco,VinylSd,Wd Sdng,WdShing
Exterior1st,,,,,,,,,,,,,,,,
AsbShng,17,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0
AsphShn,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BrkComm,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0
BrkFace,1,0,0,24,0,0,3,0,0,0,6,2,1,0,12,1
CBlock,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
CemntBd,0,0,0,0,0,59,0,0,0,0,0,0,0,0,1,1
HdBoard,0,1,0,0,0,0,193,2,1,0,23,0,0,0,1,1
ImStucc,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
MetalSd,0,1,0,0,0,0,3,0,212,0,0,0,1,0,2,1


All correction were made in one round.  
Now it's time to deal with meaningful NANs

In [22]:
print_nans_stat(df)

Column 	NANs
None
LotFrontage 	259
Alley 	1369
MasVnrType 	8
MasVnrArea 	8
BsmtQual 	37
BsmtCond 	37
BsmtExposure 	38
BsmtFinType1 	37
BsmtFinType2 	38
Electrical 	1
FireplaceQu 	690
GarageType 	81
GarageYrBlt 	81
GarageFinish 	81
GarageQual 	81
GarageCond 	81
PoolQC 	1453
Fence 	1179
MiscFeature 	1406
-------------------- 
Total 	6965


### Step 4 - Replace meaningful NANs in single columns

In [23]:
# Replacing nans in single columns
df = plc.DFSimpleNanReplacer(columns=single_features, default_values=default_nan_replacement).fit_transform(df)
print_nans_stat(df)

Column 	NANs
None
LotFrontage 	259
MasVnrType 	8
MasVnrArea 	8
BsmtQual 	37
BsmtCond 	37
BsmtExposure 	38
BsmtFinType1 	37
BsmtFinType2 	38
FireplaceQu 	690
GarageType 	81
GarageYrBlt 	81
GarageFinish 	81
GarageQual 	81
GarageCond 	81
PoolQC 	1453
MiscFeature 	1406
-------------------- 
Total 	4416


### Step 5 - Replace meaningful NANs in chained columns

In [24]:
df = plc.DFChainedNanReplacer(chained_features=chained_features, default_values=default_nan_replacement).fit_transform(df)
print_nans_stat(df)

Column 	NANs
None
LotFrontage 	259
BsmtExposure 	1
BsmtFinType2 	1
-------------------- 
Total 	261


In [25]:
df[df['BsmtExposure'].isna() | df['BsmtFinType2'].isna()][['BsmtExposure', 'BsmtFinType2']]

,BsmtExposure,BsmtFinType2
Id,,
333,No,NaN
949,NaN,Unf


### Step 6 - Impute single values

In [26]:
df = plc.DFSimpleImputer(col_strategy={'BsmtExposure': 'most_frequent', 'BsmtFinType2': 'most_frequent'}).fit_transform(df)

In [27]:
df[df['BsmtExposure'].isna() | df['BsmtFinType2'].isna()][['BsmtExposure', 'BsmtFinType2']]

,BsmtExposure,BsmtFinType2
Id,,


In [28]:
df.loc[[333, 949], ['BsmtExposure', 'BsmtFinType2']]

,BsmtExposure,BsmtFinType2
Id,,
333,No,Unf
949,No,Unf


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 77 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   object 
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Alley          1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   LotConfig      1460 non-null   object 
 8   LandSlope      1460 non-null   object 
 9   Neighborhood   1460 non-null   object 
 10  Condition1     1460 non-null   object 
 11  Condition2     1460 non-null   object 
 12  BldgType       1460 non-null   object 
 13  HouseStyle     1460 non-null   object 
 14  OverallQual    1460 non-null   int64  
 15  OverallCond    1460 non-null   int64  
 16  YearBuilt      1460 non-null   int64  
 17  YearRemodAdd   1460 non-null   int64  
 18  RoofStyl

### Step 7 - Create additional 'Has' features for Basement, Garage and Remodelling

In [30]:
df = plc.DFCreateAdditionalFeatures(create_bsmt=True, create_garage=True, create_remodeled=True).fit_transform(df)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   object 
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Alley          1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   LotConfig      1460 non-null   object 
 8   LandSlope      1460 non-null   object 
 9   Neighborhood   1460 non-null   object 
 10  Condition1     1460 non-null   object 
 11  Condition2     1460 non-null   object 
 12  BldgType       1460 non-null   object 
 13  HouseStyle     1460 non-null   object 
 14  OverallQual    1460 non-null   int64  
 15  OverallCond    1460 non-null   int64  
 16  YearBuilt      1460 non-null   int64  
 17  YearRemodAdd   1460 non-null   int64  
 18  RoofStyl

### Step 8 - Create additional 'Age' features for House, Remodel and Garage

In [32]:
df = plc.DFCalcAge(columns={'YearBuilt': 'AgeBuilt', 'YearRemodAdd': 'AgeRemodeled', 'GarageYrBlt': 'AgeGarage'}, calc_age_to=2022, drop_originals=False).fit_transform(df)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 83 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   object 
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Alley          1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   LotConfig      1460 non-null   object 
 8   LandSlope      1460 non-null   object 
 9   Neighborhood   1460 non-null   object 
 10  Condition1     1460 non-null   object 
 11  Condition2     1460 non-null   object 
 12  BldgType       1460 non-null   object 
 13  HouseStyle     1460 non-null   object 
 14  OverallQual    1460 non-null   int64  
 15  OverallCond    1460 non-null   int64  
 16  YearBuilt      1460 non-null   int64  
 17  YearRemodAdd   1460 non-null   int64  
 18  RoofStyl

### Step 9 - Add 'DaysSinceSold' feature

In [34]:
df = plc.DFJoinDates(day_col=None, month_col='MoSold', year_col='YrSold', calc_period_to=datetime(day=1, month=1, year=2022), new_column_name='DaysSinceSold', drop_originals=False).fit_transform(df)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 84 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   object 
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Alley          1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   LotConfig      1460 non-null   object 
 8   LandSlope      1460 non-null   object 
 9   Neighborhood   1460 non-null   object 
 10  Condition1     1460 non-null   object 
 11  Condition2     1460 non-null   object 
 12  BldgType       1460 non-null   object 
 13  HouseStyle     1460 non-null   object 
 14  OverallQual    1460 non-null   int64  
 15  OverallCond    1460 non-null   int64  
 16  YearBuilt      1460 non-null   int64  
 17  YearRemodAdd   1460 non-null   int64  
 18  RoofStyl

### Step 10 - Create One-Hot features for combined columns

First try not to convert categorical columns into pd.CategoricalDtype and process them as they are (object)

In [36]:
df = plc.DFOneHotCategoriesCombined(
    features_kits=[
            (['Condition1', 'Condition2'], [x for x in cat_unordered_features['Condition1'].categories if x not in ['Abs', 'Norm']]),
            (['Exterior1st', 'Exterior2nd'], list(cat_unordered_features['Exterior1st'].categories))
        ],
        drop_originals=True
).fit_transform(df)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Columns: 105 entries, MSSubClass to WdShing
dtypes: bool(3), float64(29), int64(35), object(38)
memory usage: 1.2+ MB


84 -> 105 columns  
84 - 4 dropped original columns + 8 one-hot columns for 'Condition*' + 17 one-hot columns for 'Exterior*' = 105 -> Ok

### Step 11 - Batch encode columns with ColumnTransformer (Single One-Hots and Ordinal)

In [38]:
onehot_categories = [list(cat_unordered_features[x].categories) for x in ohe_single_columns]
onehot_categories

[['Abs',
  '20',
  '30',
  '40',
  '45',
  '50',
  '60',
  '70',
  '75',
  '80',
  '85',
  '90',
  '120',
  '150',
  '160',
  '180',
  '190'],
 ['Abs', 'A', 'C (all)', 'FV', 'I', 'RH', 'RL', 'RP', 'RM'],
 ['Abs', 'Grvl', 'Pave'],
 ['Abs', 'Lvl', 'Bnk', 'HLS', 'Low'],
 ['Abs', 'Inside', 'Corner', 'CulDSac', 'FR2', 'FR3'],
 ['Abs',
  'Blmngtn',
  'Blueste',
  'BrDale',
  'BrkSide',
  'ClearCr',
  'CollgCr',
  'Crawfor',
  'Edwards',
  'Gilbert',
  'IDOTRR',
  'MeadowV',
  'Mitchel',
  'NAmes',
  'NoRidge',
  'NPkVill',
  'NridgHt',
  'NWAmes',
  'OldTown',
  'SWISU',
  'Sawyer',
  'SawyerW',
  'Somerst',
  'StoneBr',
  'Timber',
  'Veenker'],
 ['Abs', '1Fam', '2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'],
 ['Abs',
  '1Story',
  '1.5Fin',
  '1.5Unf',
  '2Story',
  '2.5Fin',
  '2.5Unf',
  'SFoyer',
  'SLvl'],
 ['Abs', 'Flat', 'Gable', 'Gambrel', 'Hip', 'Mansard', 'Shed'],
 ['Abs',
  'ClyTile',
  'CompShg',
  'Membran',
  'Metal',
  'Roll',
  'Tar&Grv',
  'WdShake',
  'WdShngl'],
 ['BrkCmn', 'BrkF

In [39]:
sum([len(x) for x in onehot_categories])

156

In [40]:
ordinal_categories = [list(cat_ordered_features[x].categories) for x in ore_columns]
ordinal_categories

[['Abs', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
 ['Abs', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
 ['Abs', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
 ['Abs', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
 ['Abs', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
 ['Abs', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
 ['Abs', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
 ['Abs', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
 ['Abs', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
 ['Abs', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
 ['Abs', 'IR3', 'IR2', 'IR1', 'Reg'],
 ['Abs', 'Sev', 'Mod', 'Gtl'],
 ['Abs', 'No', 'Mn', 'Av', 'Gd'],
 ['Abs', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
 ['Abs', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
 ['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'],
 ['Abs', 'Unf', 'RFn', 'Fin'],
 ['N', 'P', 'Y'],
 ['Abs', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv']]

In [41]:
sum([len(x) for x in ordinal_categories])

108

In [42]:
# Transformer for categorical columns which have to be processed separately from each other (combined columns are processed in custom DFOneHotCategoriesCombined class)
ct_cat_encoder = ColumnTransformer(
    transformers=[
        ('ohe_unordered', OneHotEncoder(categories=onehot_categories, handle_unknown='ignore'), ohe_single_columns),
        ('ore_ordered', OrdinalEncoder(categories=ordinal_categories), ore_columns), #'use_encoded_value'
    ],
    remainder='passthrough'
)

In [43]:
df = ct_cat_encoder.fit_transform(df)

In [44]:
df.shape

(1460, 242)

In [45]:
print(len(ohe_single_columns))
print(len(ore_columns))

19
19


105 columns -> 242 columns  
105 original - 19 one-hot encoded (19 ordinal encoded stays in dataset) + 156 new one-hot columns = 242 columns - Ok

### Step 12 - Impute NANs left with Imputer

In [46]:
knn_imputer = KNNImputer(n_neighbors=3)

In [47]:
df = knn_imputer.fit_transform(df)

In [48]:
df.shape

(1460, 242)

In [49]:
np.count_nonzero(np.isnan(df))

0

## Testing result dataset on RandomForest model

Seems our data processing flow results in dataset applicable for using with sklearn models. Let's check it

In [50]:
randforest_reg = RandomForestRegressor(n_estimators=150, random_state=42)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

In [52]:
randforest_reg.fit(X_train, y_train)

RandomForestRegressor(n_estimators=150, random_state=42)

In [53]:
randforest_reg.score(X_test, y_test)

0.891091840720255

In [54]:
baseline_error = mean_squared_log_error(y_test, randforest_reg.predict(X_test))
baseline_error

0.02269480647722746

## Final basic data processing flow

For using following pipeline we have to copy-and-paste all linked definitions (features aand categories lists, ColumnTransformer, etc.)

In [55]:
processing_pipeline = Pipeline([
    ('01_drop', plc.DFDropColumns(columns=['SalePrice', 'Utilities', 'Street'])),
    ('02_cat_to_str', plc.DFApplyFuncToColumns(columns={'MSSubClass': str})),
    ('03_fix_mistypes', plc.DFValuesReplacer(replaces={'Exterior2nd': {'CmentBd': 'CemntBd', 'Brk Cmn': 'BrkComm', 'Wd Shng': 'WdShing'}})),
    ('04_smpl_nans', plc.DFSimpleNanReplacer(columns=single_features, default_values=default_nan_replacement)),
    ('05_chnd_nans', plc.DFChainedNanReplacer(chained_features=chained_features, default_values=default_nan_replacement)),
    ('06_smpl_impute', plc.DFSimpleImputer(col_strategy={
        'BsmtExposure': 'most_frequent', 'BsmtFinType2': 'most_frequent', 'BsmtCond': 'most_frequent', 'KitchenQual': 'most_frequent', 'GarageQual': 'most_frequent',
        'GarageCond': 'most_frequent', 'Functional': 'most_frequent', 'GarageFinish': 'most_frequent'
    })),
    ('07_add_has_feats', plc.DFCreateAdditionalFeatures(create_bsmt=True, create_garage=True, create_remodeled=True)),
    ('08_add_age_feats', plc.DFCalcAge(columns={'YearBuilt': 'AgeBuilt', 'YearRemodAdd': 'AgeRemodeled', 'GarageYrBlt': 'AgeGarage'}, calc_age_to=2022, drop_originals=False)),
    ('09_add_days_feat', plc.DFJoinDates(day_col=None, month_col='MoSold', year_col='YrSold', calc_period_to=datetime(day=1, month=1, year=2022), new_column_name='DaysSinceSold', drop_originals=False)),
    ('10_ohe_chnd', plc.DFOneHotCategoriesCombined(
        features_kits=[
                (['Condition1', 'Condition2'], [x for x in cat_unordered_features['Condition1'].categories if x not in ['Abs', 'Norm']]),
                (['Exterior1st', 'Exterior2nd'], list(cat_unordered_features['Exterior1st'].categories))
            ],
            drop_originals=True
    )),
    ('11_btch_ct', ct_cat_encoder),
    ('12_impute', KNNImputer(n_neighbors=3))
])

Let's test this pipe line one more time in solid state

In [56]:
df = df_orig.copy()

In [57]:
df_processed = processing_pipeline.fit_transform(df)

In [58]:
df_processed.shape

(1460, 242)

And save resulting numpy array

In [59]:
np.savetxt("datasets/processed/train_numpy.csv", df_processed, delimiter=",")

In [269]:
processing_pipeline_pt1 = Pipeline([
    ('01_drop', plc.DFDropColumns(columns=['SalePrice', 'Utilities', 'Street'])),
    ('02_cat_to_str', plc.DFApplyFuncToColumns(columns={'MSSubClass': str})),
    ('03_fix_mistypes', plc.DFValuesReplacer(replaces={'Exterior2nd': {'CmentBd': 'CemntBd', 'Brk Cmn': 'BrkComm', 'Wd Shng': 'WdShing'}})),
    ('04_smpl_nans', plc.DFSimpleNanReplacer(columns=single_features, default_values=default_nan_replacement)),
    ('05_chnd_nans', plc.DFChainedNanReplacer(chained_features=chained_features, default_values=default_nan_replacement)),
    ('06_smpl_impute', plc.DFSimpleImputer(col_strategy={
        'BsmtExposure': 'most_frequent', 'BsmtFinType2': 'most_frequent', 'BsmtCond': 'most_frequent', 'KitchenQual': 'most_frequent', 'GarageQual': 'most_frequent',
        'GarageCond': 'most_frequent', 'Functional': 'most_frequent', 'GarageFinish': 'most_frequent'
    })),
    ('07_add_has_feats', plc.DFCreateAdditionalFeatures(create_bsmt=True, create_garage=True, create_remodeled=True)),
    ('08_add_age_feats', plc.DFCalcAge(columns={'YearBuilt': 'AgeBuilt', 'YearRemodAdd': 'AgeRemodeled', 'GarageYrBlt': 'AgeGarage'}, calc_age_to=2022, drop_originals=False)),
    ('09_add_days_feat', plc.DFJoinDates(day_col=None, month_col='MoSold', year_col='YrSold', calc_period_to=datetime(day=1, month=1, year=2022), new_column_name='DaysSinceSold', drop_originals=False)),
    ('10_ohe_chnd', plc.DFOneHotCategoriesCombined(
        features_kits=[
                (['Condition1', 'Condition2'], [x for x in cat_unordered_features['Condition1'].categories if x not in ['Abs', 'Norm']]),
                (['Exterior1st', 'Exterior2nd'], list(cat_unordered_features['Exterior1st'].categories))
            ],
            drop_originals=True
    )),
])

processing_pipeline_pt2 = Pipeline([
    ('11_btch_ct', ct_cat_encoder),
    ('12_impute', KNNImputer(n_neighbors=3))
])

And it's done

In [60]:
df_test_processed = processing_pipeline.transform(df_test)
df_test_processed.shape

(1459, 242)

In [62]:
np.savetxt("datasets/processed/test_numpy.csv", df_test_processed, delimiter=",")